In [2]:
import pandas as pd
import csv
import numpy as np
from pathlib import Path

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
%matplotlib inline



Bad key savefig.frameon in file C:\Users\avile\anaconda3\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file C:\Users\avile\anaconda3\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file C:\Users\avile\anaconda3\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  # a log filename, sys.stdout or sys.stderr')
You probably need to get 

In [2]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
# Select the dictionary for the Celtics, which contains their team ID
celtics = [team for team in nba_teams if team['abbreviation'] == 'PHX'][0]
celtics_id = celtics['id']

from nba_api.stats.endpoints import leaguegamefinder

# Query for games where the Celtics were playing
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id)
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.groupby(games.SEASON_ID.str[-4:])[['GAME_ID']].count().loc['2020':]

# Subset the games to when the last 4 digits of SEASON_ID were 2017.
games_2021 = games[games.SEASON_ID.str[-4:] == '2020']
games_2021.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42020,1610612756,PHX,Phoenix Suns,0042000315,2021-06-28,PHX vs. LAC,None,0,5,...,NaN,0,2,2,2,0,1,2,1,-13.0
1,42020,1610612756,PHX,Phoenix Suns,0042000314,2021-06-26,PHX @ LAC,W,239,84,...,0.857,17,42,59,16,4,8,14,27,4.0
2,42020,1610612756,PHX,Phoenix Suns,0042000313,2021-06-24,PHX @ LAC,L,240,92,...,0.923,11,32,43,24,5,3,10,19,-14.0
3,42020,1610612756,PHX,Phoenix Suns,0042000312,2021-06-22,PHX vs. LAC,W,239,104,...,0.889,7,32,39,24,5,3,9,20,1.0
4,42020,1610612756,PHX,Phoenix Suns,0042000311,2021-06-20,PHX vs. LAC,W,240,120,...,1.000,7,36,43,31,5,3,7,16,6.0


In [3]:
from basketball_reference_scraper.seasons import get_schedule

ModuleNotFoundError: No module named 'basketball_reference_scraper'

In [ ]:
playoff2020 = get_schedule(2020, playoffs = True)

In [ ]:
playoff2020

In [ ]:
playoff2019 = get_schedule(2019, playoffs = True)
playoff2019

In [ ]:
np.savetxt('2020playoff.csv', playoff2020, delimiter =",", fmt ="% s")

In [124]:
csv_path = Path('2020data.csv')
df_2020 = pd.read_csv(csv_path)
df_2020.head()

,Season,Time,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
0,(2019-20),8/17/2020 0:00,UTA,DEN,125,135,-10,40.1,85.1,0.47,...,10.08,32.56,42.64,25.37,7.19,4.44,14.26,20.47,109.18,160
1,(2019-20),8/17/2020 0:00,BRO,TOR,110,134,-24,40.4,90.3,0.45,...,10.92,35.14,46.06,25.57,7.22,5.49,16.83,20.36,106.51,450
2,(2019-20),8/17/2020 0:00,PHI,BOS,101,109,-8,41.1,87.9,0.47,...,10.32,34.01,44.33,22.40,7.14,5.50,15.24,20.65,107.35,230
3,(2019-20),8/17/2020 0:00,DAL,LAC,110,118,-8,41.7,90.3,0.46,...,10.71,34.60,45.31,23.74,7.50,4.69,14.07,22.88,109.90,285
4,(2019-20),8/18/2020 0:00,ORL,MIL,122,110,12,39.3,88.6,0.44,...,9.49,36.32,45.81,24.16,7.37,4.47,14.11,21.73,108.59,900


In [125]:
# df_2020.set_index('Time', inplace=True)

df_2020['matches'] = df_2020['VTeam']+df_2020['Hteam']
u_id = []
for i, r in df_2020.iterrows():
    if r.loc['matches'][0:3] < r.loc['matches'][3:7]:
        u_id_ = r.matches[0:3] + r.matches[3:7]
        u_id.append(u_id_)
    elif r.loc['matches'][0:3] > r.loc['matches'][3:7]:
        u_id_ = r.matches[3:7] + r.matches[0:3]
        u_id.append(u_id_)
#         print(False, r['u_id'])
df_2020['u_id'] = u_id

In [126]:
def ss(df_2020):
    if df_2020['Mar.'] < 0 :
        return df_2020['Hteam']
    else:
        return df_2020['VTeam']
df_2020['winner'] = df_2020.apply(ss,axis=1)

In [127]:
MILMIA = ['DENUTA', 'UTADEN']
df = df_2020[df_2020['matches'].isin(MILMIA)]

df_2020['H_series_wins'] = 0 
df_2020['A_series_wins'] = 0

def series_score(df):
    if df['VTeam'] == df['winner']:
        return df.A_series_wins.all() + 1
    return df.H_series_wins.all() + 1

df_2020 = df_2020.groupby('u_id').apply(series_score(df_2020))

In [141]:
df_2020['Time'] = pd.to_datetime(df_2020['Time']).dt.date
df_2020 = df_2020.sort_values(["u_id", "Time"])
df_2020.reset_index()
df_2020['match'] = df_2020.u_id.eq(df_2020.u_id.shift())

In [144]:
team_one = []
team_two = []

for i, r in df_2020.iterrows():
    if r['match'] == False:
        if r.loc['u_id'][0:3] == r.loc['winner']:
            team_one_ = 1
            team_two_ = 0
            team_one.append(team_one_)
            team_two.append(team_two_)
            
        elif r.loc['u_id'][3:7] == r.loc['winner']:
            team_one_ = 0
            team_two_ = 1
            team_one.append(team_one_)
            team_two.append(team_two_)
               
    elif r.loc['match'] == True:
        if r.loc['u_id'][0:3] == r.loc['winner']:
            team_one_ = team_one[-1] + 1
            team_two_ = team_two[-1]
            team_one.append(team_one_)
            team_two.append(team_two_)
        elif r.loc['u_id'][3:7] == r.loc['winner']:
            team_one_ = team_one[-1]
            team_two_ = team_two[-1] + 1
            team_one.append(team_one_)
            team_two.append(team_two_)
            
df_2020['team_one'] = team_one
df_2020['team_two'] = team_two

df_2020.head(20)

,Season,Time,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,...,OPTO,OPPF,OPPTS,H Betting Line,matches,u_id,winner,match,team_one,team_two
65,(2019-20),2020-09-15,MIA,BOS,117,114,3,39.5,84.4,0.47,...,15.24,20.65,107.35,115,MIABOS,BOSMIA,MIA,False,0,1
67,(2019-20),2020-09-17,MIA,BOS,106,101,5,39.5,84.4,0.47,...,15.24,20.65,107.35,135,MIABOS,BOSMIA,MIA,True,0,2
69,(2019-20),2020-09-19,BOS,MIA,117,106,11,41.3,89.6,0.46,...,14.00,21.70,109.10,-165,BOSMIA,BOSMIA,BOS,True,1,2
72,(2019-20),2020-09-23,BOS,MIA,109,112,-3,41.3,89.6,0.46,...,14.00,21.70,109.10,-165,BOSMIA,BOSMIA,MIA,True,1,3
74,(2019-20),2020-09-25,MIA,BOS,108,121,-13,39.5,84.4,0.47,...,15.24,20.65,107.35,140,MIABOS,BOSMIA,BOS,True,2,3
76,(2019-20),2020-09-27,BOS,MIA,113,125,-12,41.3,89.6,0.46,...,14.00,21.70,109.10,-150,BOSMIA,BOSMIA,MIA,True,2,4
2,(2019-20),2020-08-17,PHI,BOS,101,109,-8,41.1,87.9,0.47,...,15.24,20.65,107.35,230,PHIBOS,BOSPHI,BOS,False,1,0
8,(2019-20),2020-08-19,PHI,BOS,101,128,-27,41.1,87.9,0.47,...,15.24,20.65,107.35,175,PHIBOS,BOSPHI,BOS,True,2,0
18,(2019-20),2020-08-21,BOS,PHI,102,94,8,41.3,89.6,0.46,...,13.82,20.45,108.36,-240,BOSPHI,BOSPHI,BOS,True,3,0
26,(2019-20),2020-08-23,BOS,PHI,110,106,4,41.3,89.6,0.46,...,13.82,20.45,108.36,-310,BOSPHI,BOSPHI,BOS,True,4,0


In [10]:
series_wins = df_2020.groupby(['u_id','winner'])[['winner']].count()
series_wins

winner
u_id   winner        
BOSMIA BOS          2
       MIA          4
BOSPHI BOS          4
BOSTOR BOS          4
       TOR          3
BROTOR TOR          4
DALLAC DAL          2
       LAC          4
DENLAC DEN          4
       LAC          3
DENLAL DEN          1
       LAL          4
DENUTA DEN          4
       UTA          3
HOULAL HOU          1
       LAL          4
HOUOCT HOU          4
       OCT          3
INDMIA MIA          4
LALMIA LAL          4
       MIA          2
LALPOR LAL          4
       POR          1
MIAMIL MIA          4
       MIL          1
MILORL MIL          4
       ORL          1

In [8]:
# Set x variable list of features 
df_2020_x_var_list = ['FGM','FGA','FG%','3FGM','3FGA','3FG%','FTM','FTA','FT%','OFF',
                      'DEF','REB','AST','STL','BLK','TO','PF','PTS','OPFGM','OPFGA',
                      'OPFG%','OP3FGM','OP3FGA','OP3FG%','OPFTM','OPFTA','OPFT%',
                      'OPOFF','OPDEF','OPREB','OPAST','OPSTL','OPBLK','OPTO','OPPF',
                      'OPPTS','H Betting Line'
                     ]

In [9]:
# Construct the X_train and y_train datasets
df_2020_x_train = df_2020[df_2020_x_var_list]
df_2020_y_train = df_2020['Mar.']

In [10]:
csv_path = Path('2021playoff.csv')
df_2021 = pd.read_csv(csv_path)
df_2021.head()

,Season,Time,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
0,(2020-21),5/22/2021 0:00,MIA,MIL,107.0,109.0,-2,39.22,83.74,0.47,...,8.04,33.46,41.50,26.32,7.90,3.97,14.07,18.93,108.07,190.0
1,(2020-21),5/22/2021 0:00,DAL,LAC,113.0,103.0,10,41.08,87.32,0.47,...,9.13,34.21,43.33,22.88,6.25,4.32,12.07,19.39,112.44,210.0
2,(2020-21),5/22/2021 0:00,BOS,BRO,93.0,104.0,-11,41.46,88.90,0.47,...,10.63,33.63,44.25,23.46,7.72,5.32,14.06,20.43,112.63,320.0
3,(2020-21),5/22/2021 0:00,POR,DEN,123.0,109.0,14,41.29,91.08,0.45,...,10.64,33.90,44.54,21.26,6.89,5.04,11.10,18.92,116.11,100.0
4,(2020-21),5/23/2021 0:00,WAS,PHI,118.0,125.0,-7,43.17,90.93,0.47,...,9.68,35.51,45.19,25.49,7.33,4.13,14.40,21.60,116.64,300.0


In [11]:
df_2021.set_index('Time', inplace=True)
df_2021

,Season,VTeam,Hteam,VScore,Hscore,Mar.,FGM,FGA,FG%,3FGM,...,OPOFF,OPDEF,OPREB,OPAST,OPSTL,OPBLK,OPTO,OPPF,OPPTS,H Betting Line
Time,,,,,,,,,,,,,,,,,,,,,
5/22/2021 0:00,(2020-21),MIA,MIL,107.0,109.0,-2,39.22,83.74,0.47,12.94,...,8.04,33.46,41.50,26.32,7.90,3.97,14.07,18.93,108.07,190.0
5/22/2021 0:00,(2020-21),DAL,LAC,113.0,103.0,10,41.08,87.32,0.47,13.81,...,9.13,34.21,43.33,22.88,6.25,4.32,12.07,19.39,112.44,210.0
5/22/2021 0:00,(2020-21),BOS,BRO,93.0,104.0,-11,41.46,88.90,0.47,13.60,...,10.63,33.63,44.25,23.46,7.72,5.32,14.06,20.43,112.63,320.0
5/22/2021 0:00,(2020-21),POR,DEN,123.0,109.0,14,41.29,91.08,0.45,15.72,...,10.64,33.90,44.54,21.26,6.89,5.04,11.10,18.92,116.11,100.0
5/23/2021 0:00,(2020-21),WAS,PHI,118.0,125.0,-7,43.17,90.93,0.47,10.18,...,9.68,35.51,45.19,25.49,7.33,4.13,14.40,21.60,116.64,300.0
5/23/2021 0:00,(2020-21),LAL,PHO,90.0,99.0,-9,40.63,86.07,0.47,11.06,...,9.65,34.58,44.24,24.65,7.81,5.36,15.21,19.13,109.54,120.0
5/23/2021 0:00,(2020-21),ATL,NYK,107.0,105.0,2,40.79,87.24,0.47,12.43,...,10.56,35.07,45.63,24.13,6.99,4.75,13.24,19.33,113.69,100.0
5/23/2021 0:00,(2020-21),MEM,UTA,112.0,109.0,3,42.85,91.78,0.47,11.15,...,11.15,35.32,46.47,26.92,9.10,5.06,13.29,18.74,113.29,315.0
5/24/2021 0:00,(2020-21),MIA,MIL,98.0,132.0,-34,39.22,83.74,0.47,12.94,...,8.04,33.46,41.50,26.32,7.90,3.97,14.07,18.93,108.07,180.0


In [ ]:
df_2021_x_var_list = ['FGM','FGA','FG%','3FGM','3FGA','3FG%','FTM','FTA','FT%','OFF',
                      'DEF','REB','AST','STL','BLK','TO','PF','PTS','OPFGM','OPFGA',
                      'OPFG%','OP3FGM','OP3FGA','OP3FG%','OPFTM','OPFTA','OPFT%',
                      'OPOFF','OPDEF','OPREB','OPAST','OPSTL','OPBLK','OPTO','OPPF',
                      'OPPTS','H Betting Line'
                     ]

In [ ]:
playoff2021_testing_start =  '5/22/2021 0:00'
playoff2021_testing_end = '6/20/2021 0:00'

In [ ]:
df_2021_test_X = df_2021[df_2021_x_var_list][playoff2021_testing_start:playoff2021_testing_end]
df_2021_test_y = df_2021['Mar.'][playoff2021_testing_start:playoff2021_testing_end]

In [4]:
# Fit a SKLearn linear regression using just the training set (X_train, Y_train):
df_2020_forest_model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)
df_2020_forest_model.fit(df_2020_x_train, df_2020_y_train)



NameError: name 'df_2020_x_train' is not defined

In [5]:
# Make a prediction of "y" values from the X_test dataset
playoff2021_predictions = df_2020_forest_model.predict(df_2021_test_X)


NameError: name 'df_2021_test_X' is not defined

In [29]:
# Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
playoff2021_results = df_2021_test_y.to_frame()
playoff2021_results["Predicted Mar."] = playoff2021_predictions
playoff2021_results.tail(20)

,Mar.,Predicted Mar.
Time,,
6/8/2021 0:00,-3,-12
6/9/2021 0:00,-25,-12
6/10/2021 0:00,-3,-12
6/10/2021 0:00,-6,-12
6/11/2021 0:00,16,8
6/11/2021 0:00,14,-2
6/12/2021 0:00,-26,-12
6/13/2021 0:00,-11,8
6/13/2021 0:00,7,8
